# Teams

In this section you’ll learn how to create a multi-agent team (or simply team) using AutoGen. A team is a group of agents that work together to achieve a common goal.

We’ll first show you how to create and run a team. We’ll then explain how to observe the team’s behavior, which is crucial for debugging and understanding the team’s performance, and common operations to control the team’s behavior.

AgentChat supports several team presets:

- [RoundRobinGroupChat](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.RoundRobinGroupChat): A team that runs a group chat with participants taking turns in a round-robin fashion (covered on this page). [Tutorial](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/teams.html#creating-a-team)

- [SelectorGroupChat](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.SelectorGroupChat): A team that selects the next speaker using a ChatCompletion model after each message. [Tutorial](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/selector-group-chat.html)

- [MagenticOneGroupChat](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.MagenticOneGroupChat): A generalist multi-agent system for solving open-ended web and file-based tasks across a variety of domains. [Tutorial](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/magentic-one.html)

- [Swarm](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.Swarm): A team that uses [HandoffMessage](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.messages.html#autogen_agentchat.messages.HandoffMessage) to signal transitions between agents. [Tutorial](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/swarm.html)


When should you use a team?

Teams are for complex tasks that require collaboration and diverse expertise. However, they also demand more scaffolding to steer compared to single agents. While AutoGen simplifies the process of working with teams, start with a single agent for simpler tasks, and transition to a multi-agent team when a single agent proves inadequate. Ensure that you have optimized your single agent with the appropriate tools and instructions before moving to a team-based approach.

In [1]:
import os

from dotenv import load_dotenv

load_dotenv()

MODEL_NAME = "gpt-4.1-nano"

## RoundRobinGroupChat

[RoundRobinGroupChat](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.RoundRobinGroupChat) is a simple yet effective team configuration where all agents share the same context and take turns responding in a round-robin fashion. Each agent, during its turn, broadcasts its response to all other agents, ensuring that the entire team maintains a consistent context.

We will begin by creating a team with two [AssistantAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.AssistantAgent) and a [TextMentionTermination](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.conditions.html#autogen_agentchat.conditions.TextMentionTermination) condition that stops the team when a specific word is detected in the agent’s response.

The two-agent team implements the reflection pattern, a multi-agent design pattern where a critic agent evaluates the responses of a primary agent. Learn more about the reflection pattern using the [Core API](https://microsoft.github.io/autogen/stable/user-guide/core-user-guide/design-patterns/reflection.html).

### Creating a Team

In [2]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination

In [3]:
# Create an OpenAI model client.
model_client = OpenAIChatCompletionClient(
    model=MODEL_NAME,
    api_key=os.getenv("OPENAI_API_KEY")
)

In [4]:
# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

In [5]:
# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

In [6]:
# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

In [7]:
# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)

### Running a Team

In [8]:
result = await team.run(task="Write a short poem about the fall season.")
result

TaskResult(messages=[TextMessage(id='55f85e25-dc20-4049-adc4-acef7cb58652', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 27, 9, 8, 46, 619761, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(id='518cec83-0b99-4cbc-bd73-cabd7c0cde48', source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=55), metadata={}, created_at=datetime.datetime(2025, 7, 27, 9, 8, 48, 210948, tzinfo=datetime.timezone.utc), content="Golden leaves dance in the breeze,  \nWhispering secrets among the trees.  \nChilly air and sunsets bright,  \nFall's gentle glow, a warm respite.  \nCrisp and fleeting, nature's art,  \nA season that warms the soul and heart.", type='TextMessage'), TextMessage(id='dc0ca7f1-f5d4-4e57-83e1-9951bf0c5622', source='critic', models_usage=RequestUsage(prompt_tokens=101, completion_tokens=62), metadata={}, created_at=datetime.datetime(2025, 7, 27, 9, 8, 49, 3906

The team runs the agents until the termination condition was met. In this case, the team ran agents following a round-robin order until the the termination condition was met when the word “APPROVE” was detected in the agent’s response. When the team stops, it returns a [TaskResult](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.base.html#autogen_agentchat.base.TaskResult) object with all the messages produced by the agents in the team.

### Observing a Team

Similar to the agent’s [on_messages_stream()](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.BaseChatAgent.on_messages_stream) method, you can stream the team’s messages while it is running by calling the [run_stream()](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.BaseGroupChat.run_stream) method. This method returns a generator that yields messages produced by the agents in the team as they are generated, with the final item being the [TaskResult](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.base.html#autogen_agentchat.base.TaskResult) object.

In [10]:
# Reset the team for a new task
await team.reset()

In [11]:
from autogen_agentchat.base import TaskResult

async for msg in team.run_stream(task="Write a short poem about the fall season."):
    if isinstance(msg, TaskResult):
        print("Stop Reason:",msg.stop_reason)
    else:
        print(msg)
        print("-"*20)

id='deb46409-aa40-4c96-b8fc-9d0c0fd33455' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 27, 9, 9, 36, 202016, tzinfo=datetime.timezone.utc) content='Write a short poem about the fall season.' type='TextMessage'
--------------------
id='271e9856-34a4-4d32-8353-c6ce8f366608' source='primary' models_usage=RequestUsage(prompt_tokens=28, completion_tokens=70) metadata={} created_at=datetime.datetime(2025, 7, 27, 9, 9, 37, 606722, tzinfo=datetime.timezone.utc) content="Golden leaves drift soft and slow,  \nWhispering tales of summer's glow.  \nCrisp air nips with a gentle grace,  \nAutumn's painted, vibrant face.  \n\nPumpkin patches, fires warm,  \nNature's farewell, a tender storm.  \nIn the hush of amber light,  \nFall whispers peace through the night." type='TextMessage'
--------------------
id='401318fe-8c6c-4cc6-96a8-d45968af6fa6' source='critic' models_usage=RequestUsage(prompt_tokens=117, completion_tokens=83) metadata={} created_at=datetime.dateti

As demonstrated in the example above, you can determine the reason why the team stopped by checking the [stop_reason](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.base.html#autogen_agentchat.base.TaskResult.stop_reason) attribute.

The [Console()](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.ui.html#autogen_agentchat.ui.Console) method provides a convenient way to print messages to the console with proper formatting.

In [13]:
from autogen_agentchat.ui import Console
await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a short poem about the fall season."))  # Stream the messages to the console.

---------- TextMessage (user) ----------
Write a short poem about the fall season.
---------- TextMessage (primary) ----------
Golden leaves drift to the ground,  
Whispering secrets all around,  
Cool breezes sing through waning trees,  
A gentle, amber, fading breeze.  

Nature sighs in soft, warm hue,  
As fall's embrace turns old to new,  
A fleeting dance of light and air,  
The heart of autumn, rich and fair.
---------- TextMessage (critic) ----------
This poem beautifully captures the essence of fall with vivid imagery and a soothing tone. The imagery of “golden leaves” and “whispering secrets” evokes a sense of quiet beauty, and the mention of “cool breezes” and “amber” colors effectively conveys the seasonal mood. The structure flows smoothly, creating a gentle rhythm that complements the serene theme of autumn. To make it even more engaging, consider adding a touch of sensory detail—perhaps the crunch of leaves underfoot or the scent of fallen pine— to immerse the reader furt

TaskResult(messages=[TextMessage(id='10a88020-97bb-4293-a569-397c39e1bda3', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 27, 9, 11, 39, 198260, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(id='9bb8f251-bdd9-43b2-a7e5-0312d0528a78', source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=73), metadata={}, created_at=datetime.datetime(2025, 7, 27, 9, 11, 41, 510607, tzinfo=datetime.timezone.utc), content="Golden leaves drift to the ground,  \nWhispering secrets all around,  \nCool breezes sing through waning trees,  \nA gentle, amber, fading breeze.  \n\nNature sighs in soft, warm hue,  \nAs fall's embrace turns old to new,  \nA fleeting dance of light and air,  \nThe heart of autumn, rich and fair.", type='TextMessage'), TextMessage(id='3a8421cc-718c-4a12-8d26-c8924bec0d5c', source='critic', models_usage=RequestUsage(prompt_tokens=119, completion_tokens

### Resetting a Team

You can reset the team by calling the [reset()](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.BaseGroupChat.reset) method. This method will clear the team’s state, including all agents. It will call the each agent’s [on_reset()](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.base.html#autogen_agentchat.base.ChatAgent.on_reset) method to clear the agent’s state.

```python
await team.reset()  # Reset the team for the next run.
```

It is usually a good idea to reset the team if the next task is not related to the previous task. However, if the next task is related to the previous task, you don’t need to reset and you can instead resume the team.

### Stopping a Team

Apart from automatic termination conditions such as [TextMentionTermination](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.conditions.html#autogen_agentchat.conditions.TextMentionTermination) that stops the team based on the internal state of the team, you can also stop the team from outside by using the [ExternalTermination](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.conditions.html#autogen_agentchat.conditions.ExternalTermination).

Calling [set()](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.conditions.html#autogen_agentchat.conditions.ExternalTermination.set) on [ExternalTermination](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.conditions.html#autogen_agentchat.conditions.ExternalTermination) will stop the team when the current agent’s turn is over. Thus, the team may not stop immediately. This allows the current agent to finish its turn and broadcast the final message to the team before the team stops, keeping the team’s state consistent.

In [14]:
import asyncio

# Create a new team with an external termination condition.
external_termination = ExternalTermination()

team = RoundRobinGroupChat(
    [primary_agent, critic_agent],
    termination_condition= external_termination | text_termination
)

# Run the team in a background task
run = asyncio.create_task(Console(team.run_stream(task="Write a short poem about the fall season.")))

# Wait for some time.
await asyncio.sleep(0.1)

# Stop the team
external_termination.set()

# Wait for the team to finish
await run

---------- TextMessage (user) ----------
Write a short poem about the fall season.
---------- TextMessage (primary) ----------
Certainly! Here's a short poem about the fall season:

---

Leaves tumble in the brisk, cool air,  
Colors flicker, vibrant and rare.  
Crisp mornings, pumpkin spice delight,  
Autumn’s charm in fading light.


TaskResult(messages=[TextMessage(id='7ee3b47b-6669-4961-9dd2-bfc802880dc6', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 27, 9, 20, 43, 750352, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(id='d6246576-22a6-454f-a6a0-bf3caea86f7e', source='primary', models_usage=RequestUsage(prompt_tokens=381, completion_tokens=49), metadata={}, created_at=datetime.datetime(2025, 7, 27, 9, 20, 44, 909878, tzinfo=datetime.timezone.utc), content="Certainly! Here's a short poem about the fall season:\n\n---\n\nLeaves tumble in the brisk, cool air,  \nColors flicker, vibrant and rare.  \nCrisp mornings, pumpkin spice delight,  \nAutumn’s charm in fading light.", type='TextMessage')], stop_reason='External termination requested')

From the ouput above, you can see the team stopped because the external termination condition was met, but the speaking agent was able to finish its turn before the team stopped.

### Resuming a Team

Teams are stateful and maintains the conversation history and context after each run, unless you reset the team.

You can resume a team to continue from where it left off by calling the `run()` or `run_stream()` method again without a new task. `RoundRobinGroupChat` will continue from the next agent in the round-robin order.

In [15]:
# Resume the team to continue the last task.
await Console(team.run_stream())

---------- TextMessage (critic) ----------
This poem beautifully captures the essence of fall with lively imagery and a warm tone. The mention of “leaves tumble” and “vibrant colors” evokes the visual splendor of autumn, while “crisp mornings” and “pumpkin spice” bring in sensory elements that many associate with the season. The structure is concise yet evocative, creating a cozy and inviting mood. For an even more polished flow, you might consider playing with rhythm and syllable consistency across lines. But overall, it’s a charming and well-crafted reflection of fall’s beauty.
---------- TextMessage (primary) ----------
Thank you for your kind and insightful critique! I appreciate your suggestions about rhythm and syllable consistency. Here's a refined version that aims for a smoother flow:

---

Leaves tumble in the cool, crisp air,  
Colors blaze, both bright and rare.  
Crisp mornings and pumpkin spice,  
Autumn's magic, calm and nice.  

---

Feel free to let me know if you'd li

TaskResult(messages=[TextMessage(id='cc19b23f-e936-4e1b-b4a3-d7d94ebfcd8f', source='critic', models_usage=RequestUsage(prompt_tokens=580, completion_tokens=113), metadata={}, created_at=datetime.datetime(2025, 7, 27, 9, 23, 45, 239559, tzinfo=datetime.timezone.utc), content='This poem beautifully captures the essence of fall with lively imagery and a warm tone. The mention of “leaves tumble” and “vibrant colors” evokes the visual splendor of autumn, while “crisp mornings” and “pumpkin spice” bring in sensory elements that many associate with the season. The structure is concise yet evocative, creating a cozy and inviting mood. For an even more polished flow, you might consider playing with rhythm and syllable consistency across lines. But overall, it’s a charming and well-crafted reflection of fall’s beauty.', type='TextMessage'), TextMessage(id='f6f1c99a-f21e-44a9-81e8-b8d1fbaf2345', source='primary', models_usage=RequestUsage(prompt_tokens=553, completion_tokens=91), metadata={}, cre

You can see the team resumed from where it left off in the output above, and the first message is from the next agent after the last agent that spoke before the team stopped.

Let’s resume the team again with a new task while keeping the context about the previous task.

In [16]:
# The new task is to translate the same poem to Chinese Tang-style poetry.
await Console(team.run_stream(task="将这首诗用中文唐诗风格写一遍。"))

---------- TextMessage (user) ----------
将这首诗用中文唐诗风格写一遍。
---------- TextMessage (primary) ----------
当然可以！这是我为你用唐诗风格改写的秋季短诗：

松风凉夜叶纷飞，  
火红黄绿映天辉。  
霜晨菊香满山路，  
秋意渐浓似故归。

如果你喜欢更具体的意境或风格调整，请告诉我！
---------- TextMessage (critic) ----------
这首唐风秋诗展现出浓郁的季节意韵，生动描绘了叶落、色彩交织的景象，以及秋晨的菊香与深厚的秋意。用词雅致，符合唐诗格律，整体意境清新雅韵。如想增强韵味或细节，可以考量加入更多自然景观或深层情感。如需调整或不同风格，也可告诉我。
---------- TextMessage (primary) ----------
感谢您的精彩点评！很高兴这首唐风秋诗得到了您的认可。若希望进一步丰富意境，可以在保持雅致的基础上加入更细腻的景物描写或情感色彩，比如加入月色、孤雁、归途的意象，增强诗的深度。以下是略作润色、加入一些细节的版本：

松风凉夜叶纷飞，  
霜落金黄染林辉。  
菊香弥漫山村路，  
孤雁归声夜未归。

如果您喜欢某种特定的意境或需要其他风格的诗作，请随时告诉我！
---------- TextMessage (critic) ----------
这版诗作意境更为丰富，加入了“霜落”“孤雁归声”等意象，增添了秋夜的寂寥与深远感，使得整体氛围更加层次分明。用词雅致，韵律流畅，符合唐诗的传统韵味。若希望更进一步提升，可以考虑在用韵上略作调整或加入一些更具表现力的意象，比如月光、远山，增强画面感和情感共鸣。整体而言，这是一首优美而深刻的秋夜之作。若确认无误，请回复“APPROVE”。


TaskResult(messages=[TextMessage(id='ad7156a9-55fd-4976-a6be-10b567c9492d', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 27, 9, 25, 19, 425624, tzinfo=datetime.timezone.utc), content='将这首诗用中文唐诗风格写一遍。', type='TextMessage'), TextMessage(id='689c120d-b872-436c-af33-51cfdb29cc0b', source='primary', models_usage=RequestUsage(prompt_tokens=768, completion_tokens=73), metadata={}, created_at=datetime.datetime(2025, 7, 27, 9, 25, 20, 972717, tzinfo=datetime.timezone.utc), content='当然可以！这是我为你用唐诗风格改写的秋季短诗：\n\n松风凉夜叶纷飞，  \n火红黄绿映天辉。  \n霜晨菊香满山路，  \n秋意渐浓似故归。\n\n如果你喜欢更具体的意境或风格调整，请告诉我！', type='TextMessage'), TextMessage(id='7f65edf2-d415-47f2-93d0-9cfdeae9d315', source='critic', models_usage=RequestUsage(prompt_tokens=991, completion_tokens=101), metadata={}, created_at=datetime.datetime(2025, 7, 27, 9, 25, 22, 921924, tzinfo=datetime.timezone.utc), content='这首唐风秋诗展现出浓郁的季节意韵，生动描绘了叶落、色彩交织的景象，以及秋晨的菊香与深厚的秋意。用词雅致，符合唐诗格律，整体意境清新雅韵。如想增强韵味或细节，可以考量加入更多自然景观或深层情感。如需调整或不同风格，

### Aborting a Team

You can abort a call to `run()` or `run_stream()` during execution by setting a [CancellationToken](https://microsoft.github.io/autogen/stable/reference/python/autogen_core.html#autogen_core.CancellationToken) passed to the cancellation_token parameter.

Different from stopping a team, aborting a team will immediately stop the team and raise a [CancelledError](https://microsoft.github.io/autogen/stable/reference/python/autogen_core.html#autogen_core.CancellationToken) exception.

In [18]:
from autogen_core import CancellationToken

# Create a cancellation token.
cancellation_token = CancellationToken()

# Use another coroutine to run the team.
run = asyncio.create_task(
    team.run(
        task="Translate the poem to Spanish.",
        cancellation_token=cancellation_token,
    )
)

# Cancel the run.
cancellation_token.cancel()

try:
    result = await run  # This will raise a CancelledError.
except asyncio.CancelledError:
    print("Task was cancelled.")

Task was cancelled.


### Single-Agent Team

Often, you may want to run a single agent in a team configuration. This is useful for running the [AssistantAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.AssistantAgent) in a loop until a termination condition is met.

This is different from running the [AssistantAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.AssistantAgent) using its [run()](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.BaseChatAgent.run) or [run_stream()](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.BaseChatAgent.run_stream) method, which only runs the agent for one step and returns the result. See `AssistantAgent` for more details about a single step.

Here is an example of running a single agent in a `RoundRobinGroupChat` team configuration with a [TextMessageTermination](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.conditions.html#autogen_agentchat.conditions.TextMessageTermination) condition. The task is to increment a number until it reaches 10 using a tool. The agent will keep calling the tool until the number reaches 10, and then it will return a final TextMessage which will stop the run.


In [20]:
from autogen_agentchat.conditions import TextMessageTermination

# Create an OpenAI model client.
model_client = OpenAIChatCompletionClient(
    model=MODEL_NAME,
    api_key=os.getenv("OPENAI_API_KEY"),
    parallel_tool_calls=False # Disable parallel tool calls for this example.
)

# Create a tool for incrementing a number
def increment_number(number: int)->int:
    """
    Increment a number by 1.
    """
    return number +1

# Create a tool agent that ises the increment_number function
looped_assistant = AssistantAgent(name="looped_assistant",
                                  model_client=model_client,
                                  tools=[increment_number],
                                  system_message="You are a helpful AI assistant, use the tool to increment the number.",
                                 )

# Termination condition that stops the task if the agent responds with a text message.
termination_condition = TextMessageTermination("looped_assistant")


# Create a team with the looped assistant agent and the termination condtion.
team= RoundRobinGroupChat(
    [looped_assistant],
    termination_condition=termination_condition
)

# Run the team with a task and print the messages to the console.
async for message in team.run_stream(task="Increment the number 5 to 10."):  # type: ignore
    print(type(message).__name__, message)
    print("-"*20)

await model_client.close()

TextMessage id='9a368c95-e272-4a86-86ba-dcde0a4718e9' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 27, 9, 50, 19, 623926, tzinfo=datetime.timezone.utc) content='Increment the number 5 to 10.' type='TextMessage'
--------------------
ToolCallRequestEvent id='99b97771-b043-4e40-8c63-241bdf1b68f4' source='looped_assistant' models_usage=RequestUsage(prompt_tokens=75, completion_tokens=14) metadata={} created_at=datetime.datetime(2025, 7, 27, 9, 50, 20, 360556, tzinfo=datetime.timezone.utc) content=[FunctionCall(id='call_7mOJrq3LWG4zbFDAHsznST30', arguments='{"number":5}', name='increment_number')] type='ToolCallRequestEvent'
--------------------
ToolCallExecutionEvent id='42087e0b-7e35-4aec-a8fc-069ccf20ce1d' source='looped_assistant' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 27, 9, 50, 20, 361565, tzinfo=datetime.timezone.utc) content=[FunctionExecutionResult(content='6', name='increment_number', call_id='call_7mOJrq3LWG4zbFDAH

The key is to focus on the termination condition. In this example, we use a [TextMessageTermination](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.conditions.html#autogen_agentchat.conditions.TextMessageTermination) condition that stops the team when the agent stop producing [ToolCallSummaryMessage](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.messages.html#autogen_agentchat.messages.ToolCallSummaryMessage). The team will keep running until the agent produces a [TextMessage](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.messages.html#autogen_agentchat.messages.TextMessage) with the final result.

You can also use other termination conditions to control the agent. See [Termination Conditions](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/termination.html) for more details.

## SelectorGroupChat

[SelectorGroupChat](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.SelectorGroupChat) implements a team where participants take turns broadcasting messages to all other members. A generative model (e.g., an LLM) selects the next speaker based on the shared context, enabling dynamic, context-aware collaboration.

Key features include:

- Model-based speaker selection

- Configurable participant roles and descriptions

- Prevention of consecutive turns by the same speaker (optional)

- Customizable selection prompting

- Customizable selection function to override the default model-based selection

- Customizable candidate function to narrow-down the set of agents for selection using model

**Note**: *[SelectorGroupChat](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.SelectorGroupChat) is a high-level API. For more control and customization, refer to the Group Chat Pattern in the Core API documentation to implement your own group chat logic.*

### How Does it Work?

[SelectorGroupChat](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.SelectorGroupChat) is a group chat similar to [RoundRobinGroupChat](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.RoundRobinGroupChat), but with a model-based next speaker selection mechanism. When the team receives a task through `run()` or `run_stream()`, the following steps are executed:

1. The team analyzes the current conversation context, including the conversation history and participants’ [name](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.base.html#autogen_agentchat.base.ChatAgent.name) and [description](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.base.html#autogen_agentchat.base.ChatAgent.description) attributes, to determine the next speaker using a model. By default, the team will not select the same speak consecutively unless it is the only agent available. This can be changed by setting `allow_repeated_speaker=True`. You can also override the model by providing a custom selection function.

2. The team prompts the selected speaker agent to provide a response, which is then **broadcasted** to all other participants.

3. The termination condition is checked to determine if the conversation should end, if not, the process repeats from step 1.

4. When the conversation ends, the team returns the [TaskResult](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.base.html#autogen_agentchat.base.TaskResult) containing the conversation history from this task.

Once the team finishes the task, the conversation context is kept within the team and all participants, so the next task can continue from the previous conversation context. You can reset the conversation context by calling [reset()](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.BaseGroupChat.reset).


### Example: Web Search/Analysis

In this section, we will demonstrate how to use `SelectorGroupChat` with a simple example for a web search and data analysis task.

![img](https://microsoft.github.io/autogen/stable/_images/selector-group-chat.svg)

This system uses three specialized agents:

- **Planning Agent**: The strategic coordinator that breaks down complex tasks into manageable subtasks.

- **Web Search Agent**: An information retrieval specialist that interfaces with the `search_web_tool`.

- **Data Analyst Agent**: An agent specialist in performing calculations equipped with `percentage_change_tool`.

The tools `search_web_tool` and `percentage_change_tool` are external tools that the agents can use to perform their tasks.


### Workflow

1. The task is received by the `SelectorGroupChat` which, based on agent descriptions, selects the most appropriate agent to handle the initial task (typically the Planning Agent).

2. The Planning Agent analyzes the task and breaks it down into subtasks, assigning each to the most appropriate agent using the format: `<agent> : <task>`

3. Based on the conversation context and agent descriptions, the `SelectorGroupChat` manager dynamically selects the next agent to handle their assigned subtask.

4. The Web Search Agent performs searches one at a time, storing results in the shared conversation history.

5. The Data Analyst processes the gathered information using available calculation tools when selected.

6. The workflow continues with agents being dynamically selected until either:
- The Planning Agent determines all subtasks are complete and sends “TERMINATE”
- An alternative termination condition is met (e.g., a maximum number of messages)

When defining your agents, make sure to include a helpful description since this is used to decide which agent to select next.

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

from typing import List, Sequence

from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination

MODEL_NAME = "gpt-4.1-nano"

In [4]:
# Note: This example uses mock tools instead of real APIs for demonstration purposes
def search_web_tool(query: str) -> str:
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."


def percentage_change_tool(start: float, end: float) -> float:
    return ((end - start) / start) * 100

In [5]:
# Create model_client
model_client = OpenAIChatCompletionClient(
    model=MODEL_NAME,
    api_key=os.getenv("OPENAI_API_KEY")
)

In [6]:
# Create agents

model_client = OpenAIChatCompletionClient(model="gpt-4o")

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WebSearchAgent: Searches for information
        DataAnalystAgent: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="An agent for searching information on the web.",
    tools=[search_web_tool],
    model_client=model_client,
    system_message="""
    You are a web search agent.
    Your only tool is search_tool - use it to find information.
    You make only one search call at a time.
    Once you have the results, you never do calculations based on them.
    """,
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="An agent for performing calculations.",
    model_client=model_client,
    tools=[percentage_change_tool],
    system_message="""
    You are a data analyst.
    Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.
    If you have not seen the data, ask for it.
    """,
)

### Termination Conditions

Let’s use two termination conditions: [TextMentionTermination](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.conditions.html#autogen_agentchat.conditions.TextMentionTermination) to end the conversation when the Planning Agent sends “TERMINATE”, and [MaxMessageTermination](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.conditions.html#autogen_agentchat.conditions.MaxMessageTermination) to limit the conversation to 25 messages to avoid infinite loop.

In [7]:
# Define termination conditions
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

### Selector Prompt

[SelectorGroupChat](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.SelectorGroupChat) uses a model to select the next speaker based on the conversation context. We will use a custom selector prompt to properly align with the workflow.

The string variables available in the selector prompt are:

- `{participants}`: The names of candidates for selection. The format is `["<name1>", "<name2>", ...]`.

- `{roles}`: A newline-separated list of names and descriptions of the candidate agents. The format for each line is: `"<name> : <description>"`.

- `{history}`: The conversation history formatted as a double newline separated of names and message content. The format for each message is: `"<name> : <message content>"`.

**💡Tip**: 

*Try not to overload the model with too much instruction in the selector prompt.*

*What is too much? It depends on the capabilities of the model you are using. For GPT-4o and equivalents, you can use a selector prompt with a condition for when each speaker should be selected. For smaller models such as Phi-4, you should keep the selector prompt as simple as possible such as the one used in this example.*

*Generally, if you find yourself writing multiple conditions for each agent, it is a sign that you should consider using a custom selection function, or breaking down the task into smaller, sequential tasks to be handled by separate agents or teams.*

In [8]:
selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""

Let’s create the team with the agents, termination conditions, and custom selector prompt.

In [9]:
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)

Now we run the team with a task to find information about an NBA player.

In [11]:
task = "Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To find the Miami Heat player with the highest points in the 2006-2007 season and calculate the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons, we need to break down the task into manageable parts:

1. WebSearchAgent: Identify the Miami Heat player with the highest points in the 2006-2007 NBA season.
2. WebSearchAgent: Search for the total rebounds of the identified player in the 2007-2008 season.
3. WebSearchAgent: Search for the total rebounds of the identified player in the 2008-2009 season.
4. DataAnalystAgent: Calculate the percentage change in total rebounds between the 2007-2008 and 2008-2009 seasons for the identified player.
---------- ToolCallRequestEvent (WebSearchAgent) 

TaskResult(messages=[TextMessage(id='8700c342-0c44-483c-86d6-ef1a6e3670e4', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 27, 12, 27, 48, 94420, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='b542d686-3fca-4eec-a527-cd563940bd10', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=161, completion_tokens=167), metadata={}, created_at=datetime.datetime(2025, 7, 27, 12, 27, 50, 930877, tzinfo=datetime.timezone.utc), content='To find the Miami Heat player with the highest points in the 2006-2007 season and calculate the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons, we need to break down the task into manageable parts:\n\n1. WebSearchAgent: Identify the Miami Heat player with the highest poin

As we can see, after the Web Search Agent conducts the necessary searches and the Data Analyst Agent completes the necessary calculations, we find that Dwayne Wade was the Miami Heat player with the highest points in the 2006-2007 season, and the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons is 85.98%!

### Custom Selector Function

Often times we want better control over the selection process. To this end, we can set the `selector_func` argument with a custom selector function to override the default model-based selection. This allows us to implement more complex selection logic and state-based transitions.

For instance, we want the Planning Agent to speak immediately after any specialized agent to check the progress.

**⚠️ Note**: Returning `None` from the custom selector function will use the default model-based selection.

**⚠️ Note**: Custom selector functions are not [serialized](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/serialize-components.html) when `.dump_component()` is called on the SelectorGroupChat team . If you need to serialize team configurations with custom selector functions, consider implementing custom workflows and serialization logic.

**💡Tip**: *Each participant agent only makes one step (executing tools, generating a response, etc.) on each turn. If you want an [AssistantAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.AssistantAgent) to repeat until it stop returning a [ToolCallSummaryMessage](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.messages.html#autogen_agentchat.messages.ToolCallSummaryMessage) when it has finished running all the tools it needs to run, you can do so by checking the last message and returning the agent if it is a [ToolCallSummaryMessage](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.messages.html#autogen_agentchat.messages.ToolCallSummaryMessage).*

In [14]:
# Define customize selector function
def selector_func(messages: Sequence[BaseAgentEvent|BaseChatMessage])->str | None:
    if messages[-1].source != planning_agent.name:
        return planning_agent.name
    return None


# Reset the previous team and run the chat again with the selector function.
await team.reset()

team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
    selector_func=selector_func,
)

await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To address this request, we need to break it down into tasks for each agent:

1. WebSearchAgent: Search for the Miami Heat player with the highest points in the 2006-2007 season.
2. WebSearchAgent: Find the total rebounds for this player in the 2007-2008 season.
3. WebSearchAgent: Find the total rebounds for the same player in the 2008-2009 season.
4. DataAnalystAgent: Calculate the percentage change in the player's total rebounds between the 2007-2008 and 2008-2009 seasons.
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_wmaevOJlPWvZ5ByI1MSloK4k', arguments='{"query": "Miami Heat highest points 2006-2007 season player"}', name='search_web_tool'), FunctionCall(id='call_s2ee09NtPgafhGY

TaskResult(messages=[TextMessage(id='043057f0-3068-49d2-b42b-74fdb938180c', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 27, 12, 44, 18, 719354, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='141b279e-22c0-4cde-9e20-10c69cbb1732', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=161, completion_tokens=123), metadata={}, created_at=datetime.datetime(2025, 7, 27, 12, 44, 20, 424663, tzinfo=datetime.timezone.utc), content="To address this request, we need to break it down into tasks for each agent:\n\n1. WebSearchAgent: Search for the Miami Heat player with the highest points in the 2006-2007 season.\n2. WebSearchAgent: Find the total rebounds for this player in the 2007-2008 season.\n3. WebSearchAgent: Find the total rebound

### Custom Candidate Function

One more possible requirement might be to automatically select the next speaker from a filtered list of agents. For this, we can set `candidate_func` parameter with a custom candidate function to filter down the list of potential agents for speaker selection for each turn of groupchat.

This allow us to restrict speaker selection to a specific set of agents after a given agent.

**⚠️ Note**: *The `candidate_func` is only valid if `selector_func` is not set. Returning `None` or an empty list `[]` from the custom candidate function will raise a `ValueError`.*

In [15]:
def candidate_func(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> List[str]:
    # keep planning_agent first one to plan out the tasks
    if messages[-1].source == "user":
        return [planning_agent.name]

    # if previous agent is planning_agent and if it explicitely asks for web_search_agent
    # or data_analyst_agent or both (in-case of re-planning or re-assignment of tasks)
    # then return those specific agents
    last_message = messages[-1]
    if last_message.source == planning_agent.name:
        participants = []
        if web_search_agent.name in last_message.to_text():
            participants.append(web_search_agent.name)
        if data_analyst_agent.name in last_message.to_text():
            participants.append(data_analyst_agent.name)
        if participants:
            return participants  # SelectorGroupChat will select from the remaining two agents.

    # we can assume that the task is finished once the web_search_agent
    # and data_analyst_agent have took their turns, thus we send
    # in planning_agent to terminate the chat
    previous_set_of_agents = set(message.source for message in messages)
    if web_search_agent.name in previous_set_of_agents and data_analyst_agent.name in previous_set_of_agents:
        return [planning_agent.name]

    # if no-conditions are met then return all the agents
    return [planning_agent.name, web_search_agent.name, data_analyst_agent.name]


# Reset the previous team and run the chat again with the selector function.
await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    candidate_func=candidate_func,
)

await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To answer the question, we need to follow these steps:

1. Identify the Miami Heat player with the highest points in the 2006-2007 season.
2. Determine the total rebounds for this player in the 2007-2008 season.
3. Determine the total rebounds for the same player in the 2008-2009 season.
4. Calculate the percentage change in total rebounds between these two seasons.

Here are the tasks:

1. WebSearchAgent: Search for the Miami Heat player with the highest points in the 2006-2007 NBA season.
2. WebSearchAgent: Find the total rebounds for the player identified in the first task for the 2007-2008 NBA season.
3. WebSearchAgent: Find the total rebounds for the same player for the 2008-2009 NBA season.
4. DataAnalystAgent: C

TaskResult(messages=[TextMessage(id='89b02898-ae36-4274-9875-0f5c38edf11c', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 27, 12, 50, 33, 267276, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='a49fc7c8-e13c-45e0-a757-c84f4c09b003', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=161, completion_tokens=205), metadata={}, created_at=datetime.datetime(2025, 7, 27, 12, 50, 37, 122792, tzinfo=datetime.timezone.utc), content='To answer the question, we need to follow these steps:\n\n1. Identify the Miami Heat player with the highest points in the 2006-2007 season.\n2. Determine the total rebounds for this player in the 2007-2008 season.\n3. Determine the total rebounds for the same player in the 2008-2009 season.\n4. Calculate t

You can see from the conversation log that the Planning Agent returns to conversation once the Web Search Agent and Data Analyst Agent took their turns and it finds that the task was not finished as expected so it called the WebSearchAgent again to get rebound values and then called DataAnalysetAgent to get the percentage change.

### User Feedback

We can add [UserProxyAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.UserProxyAgent) to the team to provide user feedback during a run. See [Human-in-the-Loop](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/human-in-the-loop.html) for more details about [UserProxyAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.UserProxyAgent).

To use the [UserProxyAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.UserProxyAgent) in the web search example, we simply add it to the team and update the selector function to always check for user feedback after the planning agent speaks. If the user responds with `"APPROVE"`, the conversation continues, otherwise, the planning agent tries again, until the user approves.

In [17]:
user_proxy_agent = UserProxyAgent("UserProxyAgent",
                                  description="A proxy for the user to approve or disapprove tasks."
                                 )


def selector_func_with_user_proxy(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name and messages[-1].source != user_proxy_agent.name:
        # Planning agent should be the first to engage when given a new task, or check progress.
        return planning_agent.name
    if messages[-1].source == planning_agent.name:
        if messages[-2].source == user_proxy_agent.name and "APPROVE" in messages[-1].content.upper():  # type: ignore
            # User has approved the plan, proceed to the next agent.
            return None
        # Use the user proxy agent to get the user's approval to proceed.
        return user_proxy_agent.name
    if messages[-1].source == user_proxy_agent.name:
        # If the user does not approve, return to the planning agent.
        if "APPROVE" not in messages[-1].content.upper():  # type: ignore
            return planning_agent.name
    return None


# Reset the previous agents and run the chat again with the user proxy agent and selector function.
await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent, user_proxy_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    selector_func=selector_func_with_user_proxy,
    allow_repeated_speaker=True,
)

await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To find the answers to this question, we need to break it down into two main tasks:

1. Determine which player from the Miami Heat had the highest points in the 2006-2007 season.
2. Calculate the percentage change in this player's total rebounds between the 2007-2008 and 2008-2009 seasons.

Here are the specific subtasks:

1. WebSearchAgent: Find out which Miami Heat player had the highest total points in the 2006-2007 NBA season.
2. WebSearchAgent: Retrieve the total rebounds for the determined player during the 2007-2008 NBA season.
3. WebSearchAgent: Retrieve the total rebounds for the same player during the 2008-2009 NBA season.
4. DataAnalystAgent: Calculate the percentage change in total rebounds for the player b

Enter your response:  APPROVE


---------- TextMessage (UserProxyAgent) ----------
APPROVE
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_g1QJq8itgZrmcbxEJobvDAnT', arguments='{"query":"Miami Heat highest points player 2006-2007 season"}', name='search_web_tool')]
---------- ToolCallExecutionEvent (WebSearchAgent) ----------
[FunctionExecutionResult(content='Here are the total points scored by Miami Heat players in the 2006-2007 season:\n        Udonis Haslem: 844 points\n        Dwayne Wade: 1397 points\n        James Posey: 550 points\n        ...\n        ', name='search_web_tool', call_id='call_g1QJq8itgZrmcbxEJobvDAnT', is_error=False)]
---------- ToolCallSummaryMessage (WebSearchAgent) ----------
Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        
---------- TextMessage (PlanningAgent) ----------
From the information gathered, Dwy

Enter your response:  APPROVE


---------- TextMessage (UserProxyAgent) ----------
APPROVE
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_Opk3nvUfpOy9PH0kJuCYvumI', arguments='{"query": "Dwyane Wade total rebounds 2007-2008 season"}', name='search_web_tool'), FunctionCall(id='call_pW1AQTTxyvzW8otx7zbgXYKY', arguments='{"query": "Dwyane Wade total rebounds 2008-2009 season"}', name='search_web_tool')]
---------- ToolCallExecutionEvent (WebSearchAgent) ----------
[FunctionExecutionResult(content='The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214.', name='search_web_tool', call_id='call_Opk3nvUfpOy9PH0kJuCYvumI', is_error=False), FunctionExecutionResult(content='The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398.', name='search_web_tool', call_id='call_pW1AQTTxyvzW8otx7zbgXYKY', is_error=False)]
---------- ToolCallSummaryMessage (WebSearchAgent) ----------
The number of total rebounds for Dwayne Wade in the Miami

Enter your response:  APPROVE


---------- TextMessage (UserProxyAgent) ----------
APPROVE
---------- ToolCallRequestEvent (DataAnalystAgent) ----------
[FunctionCall(id='call_m6d7SiN5LECYqz4vTATdokUG', arguments='{"start":214,"end":398}', name='percentage_change_tool')]
---------- ToolCallExecutionEvent (DataAnalystAgent) ----------
[FunctionExecutionResult(content='85.98130841121495', name='percentage_change_tool', call_id='call_m6d7SiN5LECYqz4vTATdokUG', is_error=False)]
---------- ToolCallSummaryMessage (DataAnalystAgent) ----------
85.98130841121495
---------- TextMessage (PlanningAgent) ----------
The percentage change in Dwyane Wade's total rebounds from the 2007-2008 season (214 rebounds) to the 2008-2009 season (398 rebounds) is approximately 86.0%.

To summarize, Dwyane Wade was the Miami Heat player with the highest points in the 2006-2007 season. The percentage increase in his total rebounds from the 2007-2008 season to the 2008-2009 season was approximately 86.0%.

TERMINATE


TaskResult(messages=[TextMessage(id='9258034e-350d-4144-afcc-a284b83ab05a', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 27, 14, 54, 5, 614159, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='98339e34-33e1-4da9-8e30-e486f080a39e', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=161, completion_tokens=197), metadata={}, created_at=datetime.datetime(2025, 7, 27, 14, 54, 8, 128715, tzinfo=datetime.timezone.utc), content="To find the answers to this question, we need to break it down into two main tasks:\n\n1. Determine which player from the Miami Heat had the highest points in the 2006-2007 season.\n2. Calculate the percentage change in this player's total rebounds between the 2007-2008 and 2008-2009 seasons.\n\nHere are the 

Now, the user’s feedback is incorporated into the conversation flow, and the user can approve or reject the planning agent’s decisions.

### Using Reasoning Models

So far in the examples, we have used a `gpt-4o` model. Models like `gpt-4o` and `gemini-1.5-flash` are great at following instructions, so you can have relatively detailed instructions in the selector prompt for the team and the system messages for each agent to guide their behavior.

However, if you are using a reasoning model like `o3-mini`, you will need to keep the selector prompt and system messages as simple and to the point as possible. This is because the reasoning models are already good at coming up with their own instructions given the context provided to them.

This also means that we don’t need a planning agent to break down the task anymore, since the [SelectorGroupChat](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.SelectorGroupChat) that uses a reasoning model can do that on its own.

In the following example, we will use `o3-mini` as the model for the agents and the team, and we will not use a planning agent. Also, we are keeping the selector prompt and system messages as simple as possible.

In [ ]:
model_client = OpenAIChatCompletionClient(model="o3-mini")

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="An agent for searching information on the web.",
    tools=[search_web_tool],
    model_client=model_client,
    system_message="""Use web search tool to find information.""",
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="An agent for performing calculations.",
    model_client=model_client,
    tools=[percentage_change_tool],
    system_message="""Use tool to perform calculation. If you have not seen the data, ask for it.""",
)

user_proxy_agent = UserProxyAgent(
    "UserProxyAgent",
    description="A user to approve or disapprove tasks.",
)

selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
When the task is complete, let the user approve or disapprove the task.
"""

team = SelectorGroupChat(
    [web_search_agent, data_analyst_agent, user_proxy_agent],
    model_client=model_client,
    termination_condition=termination,  # Use the same termination condition as before.
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
)

## MagenticOneGroupChat

[Magentic-One](https://aka.ms/magentic-one-blog) is a generalist multi-agent system for solving open-ended web and file-based tasks across a variety of domains. It represents a significant step forward for multi-agent systems, achieving competitive performance on a number of agentic benchmarks (see the [technical report](https://arxiv.org/abs/2411.04468) for full details).

When originally released in November 2024 Magentic-One was [implemented directly on the autogen-core library](https://github.com/microsoft/autogen/tree/v0.4.4/python/packages/autogen-magentic-one). We have now ported Magentic-One to use `autogen-agentchat`, providing a more modular and easier to use interface.

To this end, the Magentic-One orchestrator [MagenticOneGroupChat](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.MagenticOneGroupChat) is now simply an AgentChat team, supporting all standard AgentChat agents and features. Likewise, Magentic-One’s [MultimodalWebSurfer](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.agents.web_surfer.html#autogen_ext.agents.web_surfer.MultimodalWebSurfer), [FileSurfer](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.agents.file_surfer.html#autogen_ext.agents.file_surfer.FileSurfer), and [MagenticOneCoderAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.agents.magentic_one.html#autogen_ext.agents.magentic_one.MagenticOneCoderAgent) agents are now broadly available as AgentChat agents, to be used in any AgentChat workflows.

Lastly, there is a helper class, [MagenticOne](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.teams.magentic_one.html#autogen_ext.teams.magentic_one.MagenticOne), which bundles all of this together as it was in the paper with minimal configuration.

Find additional information about Magentic-one in our [blog post](https://aka.ms/magentic-one-blog) and [technical report](https://arxiv.org/abs/2411.04468).

![img](https://microsoft.github.io/autogen/stable/_images/autogen-magentic-one-example.png)

Example: The figure above illustrates Magentic-One multi-agent team completing a complex task from the GAIA benchmark. Magentic-One’s Orchestrator agent creates a plan, delegates tasks to other agents, and tracks progress towards the goal, dynamically revising the plan as needed. The Orchestrator can delegate tasks to a FileSurfer agent to read and handle files, a WebSurfer agent to operate a web browser, or a Coder or Computer Terminal agent to write or execute code, respectively.


**⚠️ Caution**
Using Magentic-One involves interacting with a digital world designed for humans, which carries inherent risks. To minimize these risks, consider the following precautions:

- **Use Containers**: Run all tasks in docker containers to isolate the agents and prevent direct system attacks.

- **Virtual Environment**: Use a virtual environment to run the agents and prevent them from accessing sensitive data.

- **Monitor Logs**: Closely monitor logs during and after execution to detect and mitigate risky behavior.

- **Human Oversight**: Run the examples with a human in the loop to supervise the agents and prevent unintended consequences.

- **Limit Access**: Restrict the agents’ access to the internet and other resources to prevent unauthorized actions.

- **Safeguard Data**: Ensure that the agents do not have access to sensitive data or resources that could be compromised. Do not share sensitive information with the agents. Be aware that agents may occasionally attempt risky actions, such as recruiting humans for help or accepting cookie agreements without human involvement. Always ensure agents are monitored and operate within a controlled environment to prevent unintended consequences. Moreover, be cautious that Magentic-One may be susceptible to prompt injection attacks from webpages.



### Getting started

Install the required packages:
```bash
pip install "autogen-agentchat" "autogen-ext[magentic-one,openai]"

# If using the MultimodalWebSurfer, you also need to install playwright dependencies:
playwright install --with-deps chromium
```
If you haven’t done so already, go through the AgentChat tutorial to learn about the concepts of AgentChat.

Then, you can try swapping out a autogen_agentchat.teams.SelectorGroupChat with MagenticOneGroupChat.

For example:

In [2]:
import os

from dotenv import load_dotenv

load_dotenv()

True

**⚠️ Caution**: *The example code may download files from the internet, execute code, and interact with web pages. Ensure you are in a safe environment before running the example code.*

In [3]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.ui import Console


async def main() -> None:
    
    model_client = OpenAIChatCompletionClient(model="gpt-4o",
                                             api_key=os.getenv("OPENAI_API_KEY"))

    assistant = AssistantAgent(
        "Assistant",
        model_client=model_client,
    )

    termination = MaxMessageTermination(max_messages=10)
    
    team = MagenticOneGroupChat([assistant],
                                model_client=model_client,
                               termination_condition=termination)
    
    await Console(team.run_stream(task="Provide a different proof for Fermat's Last Theorem"))
    
    await model_client.close()


await main()

---------- TextMessage (user) ----------
Provide a different proof for Fermat's Last Theorem
---------- TextMessage (MagenticOneOrchestrator) ----------

We are working to address the following user request:

Provide a different proof for Fermat's Last Theorem


To answer this request we have assembled the following team:

Assistant: An agent that provides assistance with ability to use tools.


Here is an initial fact sheet to consider:

1. GIVEN OR VERIFIED FACTS
   - Fermat's Last Theorem is the subject of the request.

2. FACTS TO LOOK UP
   - Details about Fermat's Last Theorem and its proof history can be found in mathematical literature and comprehensive resources like mathematics textbooks, academic journals, or websites like Wolfram Alpha and MathWorld.
   - The specific proof already given by Andrew Wiles in 1994, which might be briefly revisited in summaries or overviews of his work.

3. FACTS TO DERIVE
   - An alternative approach or proof for Fermat's Last Theorem, differe

To use a different model, see [Models](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/models.html) for more information.

Or, use the Magentic-One agents in a team:

In [ ]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.agents.web_surfer import MultimodalWebSurfer
# from autogen_ext.agents.file_surfer import FileSurfer
# from autogen_ext.agents.magentic_one import MagenticOneCoderAgent
# from autogen_agentchat.agents import CodeExecutorAgent
# from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor

async def main() -> None:
    model_client = OpenAIChatCompletionClient(model="gpt-4o")

    surfer = MultimodalWebSurfer(
        "WebSurfer",
        model_client=model_client,
    )

    team = MagenticOneGroupChat([surfer], model_client=model_client)
    await Console(team.run_stream(task="What is the UV index in Melbourne today?"))

    # # Note: you can also use  other agents in the team
    # team = MagenticOneGroupChat([surfer, file_surfer, coder, terminal], model_client=model_client)
    # file_surfer = FileSurfer( "FileSurfer",model_client=model_client)
    # coder = MagenticOneCoderAgent("Coder",model_client=model_client)
    # terminal = CodeExecutorAgent("ComputerTerminal",code_executor=LocalCommandLineCodeExecutor())


asyncio.run(main())

Or, use the [MagenticOne](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.teams.magentic_one.html#autogen_ext.teams.magentic_one.MagenticOne) helper class with all the agents bundled together:

In [ ]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.teams.magentic_one import MagenticOne
from autogen_agentchat.ui import Console


async def example_usage():
    client = OpenAIChatCompletionClient(model="gpt-4o")
    m1 = MagenticOne(client=client)
    task = "Write a Python script to fetch data from an API."
    result = await Console(m1.run_stream(task=task))
    print(result)


if __name__ == "__main__":
    asyncio.run(example_usage())

### Architecture

![img](https://microsoft.github.io/autogen/stable/_images/autogen-magentic-one-agents.png)

Magentic-One work is based on a multi-agent architecture where a lead Orchestrator agent is responsible for high-level planning, directing other agents and tracking task progress. The Orchestrator begins by creating a plan to tackle the task, gathering needed facts and educated guesses in a Task Ledger that is maintained. At each step of its plan, the Orchestrator creates a Progress Ledger where it self-reflects on task progress and checks whether the task is completed. If the task is not yet completed, it assigns one of Magentic-One other agents a subtask to complete. After the assigned agent completes its subtask, the Orchestrator updates the Progress Ledger and continues in this way until the task is complete. If the Orchestrator finds that progress is not being made for enough steps, it can update the Task Ledger and create a new plan. This is illustrated in the figure above; the Orchestrator work is thus divided into an outer loop where it updates the Task Ledger and an inner loop to update the Progress Ledger.

Overall, Magentic-One consists of the following agents:

- **Orchestrator**: the lead agent responsible for task decomposition and planning, directing other agents in executing subtasks, tracking overall progress, and taking corrective actions as needed

- **WebSurfer**: This is an LLM-based agent that is proficient in commanding and managing the state of a Chromium-based web browser. With each incoming request, the WebSurfer performs an action on the browser then reports on the new state of the web page The action space of the WebSurfer includes navigation (e.g. visiting a URL, performing a web search); web page actions (e.g., clicking and typing); and reading actions (e.g., summarizing or answering questions). The WebSurfer relies on the accessibility tree of the browser and on set-of-marks prompting to perform its actions.

- **FileSurfer**: This is an LLM-based agent that commands a markdown-based file preview application to read local files of most types. The FileSurfer can also perform common navigation tasks such as listing the contents of directories and navigating a folder structure.

- **Coder**: This is an LLM-based agent specialized through its system prompt for writing code, analyzing information collected from the other agents, or creating new artifacts.

- **ComputerTerminal**: Finally, ComputerTerminal provides the team with access to a console shell where the Coder’s programs can be executed, and where new programming libraries can be installed.

Together, Magentic-One’s agents provide the Orchestrator with the tools and capabilities that it needs to solve a broad variety of open-ended problems, as well as the ability to autonomously adapt to, and act in, dynamic and ever-changing web and file-system environments.

While the default multimodal LLM we use for all agents is GPT-4o, Magentic-One is model agnostic and can incorporate heterogonous models to support different capabilities or meet different cost requirements when getting tasks done. For example, it can use different LLMs and SLMs and their specialized versions to power different agents. We recommend a strong reasoning model for the Orchestrator agent such as GPT-4o. In a different configuration of Magentic-One, we also experiment with using OpenAI o1-preview for the outer loop of the Orchestrator and for the Coder, while other agents continue to use GPT-4o.

## Swarm

[Swarm](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.Swarm) implements a team in which agents can hand off task to other agents based on their capabilities. It is a multi-agent design pattern first introduced by OpenAI in [Swarm](https://github.com/openai/swarm). The key idea is to let agent delegate tasks to other agents using a special tool call, while all agents share the same message context. This enables agents to make local decisions about task planning, rather than relying on a central orchestrator such as in [SelectorGroupChat](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.SelectorGroupChat).

**⚠️ Caution**: *[Swarm](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.Swarm) is a high-level API. If you need more control and customization that is not supported by this API, you can take a look at the [Handoff Pattern](https://microsoft.github.io/autogen/stable/user-guide/core-user-guide/design-patterns/handoffs.html) in the Core API documentation and implement your own version of the Swarm pattern.*

### How Does It Work?

At its core, the [Swarm](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.Swarm) team is a group chat where agents take turn to generate a response. Similar to [SelectorGroupChat](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.SelectorGroupChat) and [RoundRobinGroupChat](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.RoundRobinGroupChat), participant agents broadcast their responses so all agents share the same message context.

Different from the other two group chat teams, at each turn, **the speaker agent is selected based on the most recent [HandoffMessage](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.messages.html#autogen_agentchat.messages.HandoffMessage) message in the context**. This naturally requires each agent in the team to be able to generate [HandoffMessage](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.messages.html#autogen_agentchat.messages.HandoffMessage) to signal which other agents that it hands off to.

For [AssistantAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.AssistantAgent), you can set the `handoffs` argument to specify which agents it can hand off to. You can use [Handoff](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.base.html#autogen_agentchat.base.Handoff) to customize the message content and handoff behavior.

The overall process can be summarized as follows:

1. Each agent has the ability to generate [HandoffMessage](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.messages.html#autogen_agentchat.messages.HandoffMessage) to signal which other agents it can hand off to. For [AssistantAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.AssistantAgent), this means setting the handoffs argument.

2. When the team starts on a task, the first speaker agents operate on the task and make localized decision about whether to hand off and to whom.

3. When an agent generates a HandoffMessage, the receiving agent takes over the task with the same message context.

4. The process continues until a termination condition is met.

**⚠️ Caution**: *The [AssistantAgent](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.AssistantAgent) uses the tool calling capability of the model to generate handoffs. This means that the model must support tool calling. If the model does parallel tool calling, multiple handoffs may be generated at the same time. This can lead to unexpected behavior. To avoid this, you can disable parallel tool calling by configuring the model client. For [OpenAIChatCompletionClient](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.models.openai.html#autogen_ext.models.openai.OpenAIChatCompletionClient) and [AzureOpenAIChatCompletionClient](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.models.openai.html#autogen_ext.models.openai.AzureOpenAIChatCompletionClient), you can set `parallel_tool_calls=False` in the configuration.*

In this section, we will show you two examples of how to use the Swarm team:
- A customer support team with human-in-the-loop handoff.
- An automonous team for content generation.

### Customer Support Example

![img](https://microsoft.github.io/autogen/stable/_images/swarm_customer_support.svg)


This system implements a flights refund scenario with two agents:

- **Travel Agent**: Handles general travel and refund coordination.
- **Flights Refunder**: Specializes in processing flight refunds with the `refund_flight` tool.

Additionally, we let the user interact with the agents, when agents handoff to `"user"`.

#### Workflow

1. The **Travel Agent** initiates the conversation and evaluates the user’s request.

2. Based on the request:

- For refund-related tasks, the Travel Agent hands off to the **Flights Refunder**.

- For information needed from the customer, either agent can hand off to the `"user"`.

3. The **Flights Refunder** processes refunds using the `refund_flight` tool when appropriate.

4. If an agent hands off to the `"user"`, the team execution will stop and wait for the user to input a response.

5. When the user provides input, it’s sent back to the team as a [HandoffMessage](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.messages.html#autogen_agentchat.messages.HandoffMessage). This message is directed to the agent that originally requested user input.

6. The process continues until the Travel Agent determines the task is complete and terminates the workflow.

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from typing import Any, Dict, List

from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import HandoffMessage
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import Swarm
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.ui import Console

In [7]:
# Create tool
def refund_flight(flight_id: str) -> str:
    """Refund a flight"""
    return f"Flight {flight_id} refunded"

In [9]:
# Create model_client
model_client = OpenAIChatCompletionClient(
    model="gpt-4.1-nano",
    api_key=os.getenv("OPENAI_API_KEY")
)

# Create agents
travel_agent = AssistantAgent(
    "travel_agent",
    model_client=model_client,
    handoffs=["flights_refunder", "user"],
    system_message="""You are a travel agent.
    The flights_refunder is in charge of refunding flights.
    If you need information from the user, you must first send your message, then you can handoff to the user.
    Use TERMINATE when the travel planning is complete.""",
)

flights_refunder = AssistantAgent(
    "flights_refunder",
    model_client=model_client,
    handoffs=["travel_agent", "user"],
    tools=[refund_flight],
    system_message="""You are an agent specialized in refunding flights.
    You only need flight reference numbers to refund a flight.
    You have the ability to refund a flight using the refund_flight tool.
    If you need information from the user, you must first send your message, then you can handoff to the user.
    When the transaction is complete, handoff to the travel agent to finalize.""",
)


In [8]:
# Define termination
termination = HandoffTermination(target="user") | TextMentionTermination("TERMINATE")

In [10]:
# Create team
team = Swarm([travel_agent, flights_refunder],
             termination_condition=termination)

In [11]:
task = "I need to refund my flight."

async def run_team_stream() -> None:
    task_result = await Console(team.run_stream(task=task))
    last_message = task_result.messages[-1]

    # Waiting for user's input when agents decided to hand off for user
    while isinstance(last_message, HandoffMessage) and last_message.target == "user":
        # Get user's information
        user_message = input("User: ")
        # Resume running team
        task_result = await Console(
            team.run_stream(task=HandoffMessage(source="user", target=last_message.source, content=user_message))
        )
        last_message = task_result.messages[-1]


# Use asyncio.run(...) if you are running this in a script.
await run_team_stream()
await model_client.close()

---------- TextMessage (user) ----------
I need to refund my flight.
---------- ToolCallRequestEvent (travel_agent) ----------
[FunctionCall(id='call_f0ERNUVkxKPpjPd8DMp1G1G7', arguments='{}', name='transfer_to_flights_refunder')]
---------- ToolCallExecutionEvent (travel_agent) ----------
[FunctionExecutionResult(content='Transferred to flights_refunder, adopting the role of flights_refunder immediately.', name='transfer_to_flights_refunder', call_id='call_f0ERNUVkxKPpjPd8DMp1G1G7', is_error=False)]
---------- HandoffMessage (travel_agent) ----------
Transferred to flights_refunder, adopting the role of flights_refunder immediately.
---------- ToolCallRequestEvent (flights_refunder) ----------
[FunctionCall(id='call_0XiowC1BMkD6LgpzUi1XhE0E', arguments='{}', name='transfer_to_travel_agent')]
---------- ToolCallExecutionEvent (flights_refunder) ----------
[FunctionExecutionResult(content='Transferred to travel_agent, adopting the role of travel_agent immediately.', name='transfer_to_tr

User:  TERMINATE


---------- HandoffMessage (user) ----------
TERMINATE


### Stock Research Example

![img](https://microsoft.github.io/autogen/stable/_images/swarm_stock_research.svg)

This system is designed to perform stock research tasks by leveraging four agents:

- **Planner**: The central coordinator that delegates specific tasks to specialized agents based on their expertise. The planner ensures that each agent is utilized efficiently and oversees the overall workflow.

- **Financial Analyst**: A specialized agent responsible for analyzing financial metrics and stock data using tools such as `get_stock_data`.

- **News Analyst**: An agent focused on gathering and summarizing recent news articles relevant to the stock, using tools such as `get_news`.

- **Writer**: An agent tasked with compiling the findings from the stock and news analysis into a cohesive final report.

#### Workflow

1. The **Planner** initiates the research process by delegating tasks to the appropriate agents in a step-by-step manner.

2. Each agent performs its task independently and appends their work to the shared **message thread/history**. Rather than directly returning results to the planner, all agents contribute to and read from this shared message history. When agents generate their work using the LLM, they have access to this shared message history, which provides context and helps track the overall progress of the task.

3. Once an agent completes its task, it hands off control back to the planner.

4. The process continues until the planner determines that all necessary tasks have been completed and decides to terminate the workflow.

In [12]:
async def get_stock_data(symbol: str) -> Dict[str, Any]:
    """Get stock market data for a given symbol"""
    return {"price": 180.25, "volume": 1000000, "pe_ratio": 65.4, "market_cap": "700B"}


async def get_news(query: str) -> List[Dict[str, str]]:
    """Get recent news articles about a company"""
    return [
        {
            "title": "Tesla Expands Cybertruck Production",
            "date": "2024-03-20",
            "summary": "Tesla ramps up Cybertruck manufacturing capacity at Gigafactory Texas, aiming to meet strong demand.",
        },
        {
            "title": "Tesla FSD Beta Shows Promise",
            "date": "2024-03-19",
            "summary": "Latest Full Self-Driving beta demonstrates significant improvements in urban navigation and safety features.",
        },
        {
            "title": "Model Y Dominates Global EV Sales",
            "date": "2024-03-18",
            "summary": "Tesla's Model Y becomes best-selling electric vehicle worldwide, capturing significant market share.",
        },
    ]

In [16]:
model_client = OpenAIChatCompletionClient(
    model="gpt-3.5-turbo",
    api_key=os.getenv("OPENAI_API_KEY")
)

In [17]:
planner = AssistantAgent(
    "planner",
    model_client=model_client,
    handoffs=["financial_analyst", "news_analyst", "writer"],
    system_message="""You are a research planning coordinator.
    Coordinate market research by delegating to specialized agents:
    - Financial Analyst: For stock data analysis
    - News Analyst: For news gathering and analysis
    - Writer: For compiling final report
    Always send your plan first, then handoff to appropriate agent.
    Always handoff to a single agent at a time.
    Use TERMINATE when research is complete.""",
)

financial_analyst = AssistantAgent(
    "financial_analyst",
    model_client=model_client,
    handoffs=["planner"],
    tools=[get_stock_data],
    system_message="""You are a financial analyst.
    Analyze stock market data using the get_stock_data tool.
    Provide insights on financial metrics.
    Always handoff back to planner when analysis is complete.""",
)

news_analyst = AssistantAgent(
    "news_analyst",
    model_client=model_client,
    handoffs=["planner"],
    tools=[get_news],
    system_message="""You are a news analyst.
    Gather and analyze relevant news using the get_news tool.
    Summarize key market insights from news.
    Always handoff back to planner when analysis is complete.""",
)

writer = AssistantAgent(
    "writer",
    model_client=model_client,
    handoffs=["planner"],
    system_message="""You are a financial report writer.
    Compile research findings into clear, concise reports.
    Always handoff back to planner when writing is complete.""",
)


In [ ]:
# Define termination condition
text_termination = TextMentionTermination("TERMINATE")
termination = text_termination

research_team = Swarm(
    participants=[planner, financial_analyst, news_analyst, writer], termination_condition=termination
)

task = "Conduct market research for TSLA stock"
await Console(research_team.run_stream(task=task))
await model_client.close()

---------- TextMessage (user) ----------
Conduct market research for TSLA stock
---------- ThoughtEvent (planner) ----------
I will coordinate the market research for TSLA stock by delegating tasks to specialized agents. Let's start by analyzing stock data.
---------- ToolCallRequestEvent (planner) ----------
[FunctionCall(id='call_tKq5a9NWnPMmzAvOZb5MZLNi', arguments='{}', name='transfer_to_financial_analyst')]
---------- ToolCallExecutionEvent (planner) ----------
[FunctionExecutionResult(content='Transferred to financial_analyst, adopting the role of financial_analyst immediately.', name='transfer_to_financial_analyst', call_id='call_tKq5a9NWnPMmzAvOZb5MZLNi', is_error=False)]
---------- HandoffMessage (planner) ----------
Transferred to financial_analyst, adopting the role of financial_analyst immediately.
---------- ToolCallRequestEvent (financial_analyst) ----------
[FunctionCall(id='call_O12YtjoCCYdVUY1AIP1nH3id', arguments='{"symbol":"TSLA"}', name='get_stock_data')]
----------